In [1]:
import pandas as pd
import numpy as np
import requests
import json
import datetime
from tvDatafeed import TvDatafeed, Interval

In [2]:
def getdata(name):
    data=tv.get_hist(symbol=name,exchange='BINANCE',interval=Interval.in_daily,n_bars=5000)
    data=data.reset_index()
    data["datetime"]=data["datetime"].dt.strftime('%Y-%m-%d')
    data2=data[(data["datetime"]>="2020-01-01")&(data["datetime"]<"2022-01-01")].copy()
    data2["datetime"] = pd.to_datetime(data2["datetime"])
    return data2

In [3]:
def getapi(api):
    response_API=requests.get(api)
    data=response_API.text
    parse_json=json.loads(data)
    df = pd.json_normalize(parse_json['values'])
    listtime=list(df['x'])
    listtime2=[]
    for i in listtime:
        listtime2.append(datetime.datetime.fromtimestamp(i).strftime('%Y-%m-%d'))
    df['datetime']=pd.Series(listtime2)
    df["datetime"] = pd.to_datetime(df["datetime"])
    df=df[(df["datetime"]>="2020-01-01")&(df["datetime"]<"2022-01-01")].copy()
    df=df.drop(columns=['x'])
    df.rename(columns={'y':'value'},inplace=True)
    df=df.drop_duplicates(subset=['datetime'])
    return df

In [4]:
tv = TvDatafeed()
bitcoi=getdata('BTCUSDT')
eth=getdata('ETHUSDT')
bnb=getdata('BNBUSDT')
ada=getdata('ADAUSDT')
bitcoi=bitcoi[["datetime","open","close"]]
bitcoi.tail(5)

you are using nologin method, data you access may be limited


,datetime,open,close
1594,2021-12-27,50701.44,47543.74
1595,2021-12-28,47543.74,46464.66
1596,2021-12-29,46464.66,47120.87
1597,2021-12-30,47120.88,46216.93
1598,2021-12-31,46216.93,47722.65


In [5]:
number_transactions=getapi('https://api.blockchain.info/charts/n-transactions?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
number_address=getapi('https://api.blockchain.info/charts/n-unique-addresses?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
transaction_second=getapi('https://api.blockchain.info/charts/transactions-per-second?timespan=4years&rollingAverage=24hours&format=json&sampled=false')
transaction_second=transaction_second.groupby(['datetime']).mean().reset_index()
total_bitcoin=getapi('https://api.blockchain.info/charts/total-bitcoins?timespan=4years&rollingAverage=24hours&format=json&sampled=true')
hash_rate=getapi('https://api.blockchain.info/charts/hash-rate?timespan=4years&rollingAverage=24hours&format=json&sampled=false')

In [6]:
bitcoin=bitcoi.merge(eth[["datetime","open"]],on='datetime',how='left',suffixes=('_bitcoin', '_eth'))
bitcoin=bitcoin.merge(bnb[["datetime","open"]],on='datetime',how='left')
bitcoin=bitcoin.merge(ada[["datetime","open"]],on='datetime',how='left',suffixes=('_bnb', '_ada'))

In [7]:
data_api=number_transactions.merge(number_address,on='datetime',how='left',suffixes=('_number_transaction', '_number_address'))
data_api=data_api.merge(transaction_second,on='datetime',how='left')
data_api=data_api.merge(total_bitcoin,on='datetime',how='left',suffixes=('_transaction_second', '_total_bitcoin'))
data_api=data_api.merge(hash_rate,on='datetime',how='left')
data_api.rename(columns={'value':'value_hash_rate'},inplace=True)

In [8]:
bitcoin=bitcoin.merge(data_api,on='datetime',how='left')
bitcoin.rename(columns={'close':'close_bitcoin'},inplace=True)
bitcoin.tail(5)

,datetime,open_bitcoin,close_bitcoin,open_eth,open_bnb,open_ada,value_number_transaction,value_number_address,value_transaction_second,value_total_bitcoin,value_hash_rate
726,2021-12-27,50701.44,47543.74,4037.22,563.7,1.516,265422.0,729388.0,2.374653,1.891200e+07,1.616823e+08
727,2021-12-28,47543.74,46464.66,3792.75,534.2,1.400,276083.0,741024.0,3.099653,1.891288e+07,1.749547e+08
728,2021-12-29,46464.66,47120.87,3630.18,514.5,1.333,245021.0,679386.0,3.171528,1.891377e+07,1.435835e+08
729,2021-12-30,47120.88,46216.93,3709.27,518.6,1.359,269540.0,703405.0,3.114583,1.891466e+07,1.797810e+08
730,2021-12-31,46216.93,47722.65,3676.22,511.5,1.308,186834.0,512154.0,3.098437,1.891555e+07,2.075325e+08


In [9]:
bitcoin.to_pickle("../data/interim/bitcoin_value_2020-now.pkl")